In [1]:
import numpy as np
import tensorflow as tf
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *
from sklearn.decomposition import PCA
from sklearn.datasets import load_diabetes
from scipy.stats import logistic
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from keras_radam.training import RAdamOptimizer

import tflib as lib
import tflib.save_images
import tflib.plot




In [2]:
# main parameters
Num_iter = 2000
N = 30000
max_grad_norm = 1000

BATCH_SIZE = 1000
BATCH_SIZE_2 = 1000
Lambda = 100.0
k=10

# additional parameters
gamma = 10.0
lr_decay = 0.5

# here we generate matrix X
dimensionality = 400
k = 10
delta = 0.1 # regulates percentage of outliers
MODE = 'neutral' # case I - neutral, case II-something else
number_of_points = 400
number_of_neurons = number_of_points

A = np.random.normal(loc=0.0, scale=1.0, size=(int(number_of_points*(1-delta)), k))
B = np.random.normal(loc=0.0, scale=1.0, size=(k, dimensionality))
if MODE == 'neutral':
    C = np.random.normal(loc=0.0, scale=1.0, size=(int(number_of_points*delta), dimensionality))
else:
    C = np.tile(np.random.normal(loc=0.0, scale=1.0, size=(1, dimensionality)), (int(number_of_points*delta), 1))
images = np.concatenate((np.matmul(A, B), C), axis = 0)
print(images.shape)
u, s, vh = np.linalg.svd(np.matmul(A, B), full_matrices=True)
D = vh[0:k]
P_correct = np.matmul(np.transpose(D), D)

u, s, vh = np.linalg.svd(images, full_matrices=True)
D = vh[0:k]
P_svd = np.matmul(np.transpose(D), D)
DIFF = P_svd - P_correct
dist = np.sqrt(np.sum(DIFF*DIFF))
print("Frobenius distance from SVD till correct %0.4f" %dist)

L1 = np.sum(np.sqrt(np.sum(np.square(images-np.matmul(images, P_correct)), axis=1)))
print("L12 distance till correct %0.4f" %L1)

Data = np.float32(np.transpose(images))

(400, 400)
Frobenius distance from SVD till correct 0.0043
L12 distance till correct 785.6658


In [3]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
Dataplace = tf.placeholder(tf.float32, shape=(dimensionality, None))

tf_data_x = gamma*tf.random_normal([BATCH_SIZE, dimensionality]) # аргументы функции
tf_data_y = tf.reduce_mean(tf.math.cos(tf.matmul(tf_data_x, Dataplace)), axis=1) # значения функции

tf_data_w = tf.placeholder(tf.float32, shape=(number_of_neurons,1))
tf_data_B = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons))
tf_data_P = tf.placeholder(tf.float32, shape=(dimensionality, dimensionality))

tf_data_second = gamma*tf.random_normal([BATCH_SIZE_2, dimensionality])
Lbd = tf.placeholder(tf.float32, shape=[], name="lambda")

w = tf.Variable(tf.random_normal([number_of_neurons,1], stddev=0.35), name="neuron_weights")
B = tf.Variable(initial_value=Data, name="weights")

prediction = tf.matmul(tf.math.cos(tf.matmul(tf_data_x, B)), tf.nn.sigmoid(w))
#penalty = tf.square(tf.reduce_sum(tf.nn.sigmoid(w))-1.0)

out_loss = tf.reduce_mean(tf.square(prediction - tf_data_y))# + C*penalty

prediction2 = tf.matmul(tf.math.cos(tf.matmul(tf_data_second, B)), tf.nn.sigmoid(w))

grad_psi = tf.reshape(tf.gradients(prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

new_part = grad_psi

tf_prediction2 = tf.matmul(tf.math.cos(tf.matmul(tf_data_second, tf_data_B)), tf.nn.sigmoid(tf_data_w))

tf_data_grad_psi = tf.reshape(tf.gradients(tf_prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

old_part = tf.matmul(tf_data_grad_psi, tf_data_P)

loss = out_loss + Lbd*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(new_part, old_part)), axis=1))

In [4]:
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), max_grad_norm)
optimizer = RAdamOptimizer(learning_rate=new_lr, beta1=0.5, beta2=0.9) #tf.train.GradientDescentOptimizer(new_lr) tf.train.AdamOptimizer(learning_rate=new_lr, beta1=0.5, beta2=0.9) RMSPropOptimizer(new_lr) 
target = optimizer.apply_gradients(zip(grads, tvars), global_step=tf.contrib.framework.get_or_create_global_step())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step


In [5]:
def sigmoid(x):
  return 1. /(1+np.exp(-x))

In [ ]:
cur_w = np.random.normal(0, 0.35, (number_of_neurons,1))
cur_B =  np.random.normal(0, 0.35, (dimensionality, number_of_neurons))
cur_P = np.zeros((dimensionality, dimensionality))

lr = 0.0001
prev_run_res = 1000000000

# default session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for s in range(0, Num_iter):

    lr = 0.001
    epoch = 0
    while lr > 0.0000001:  # iterations epoch < 500000 and 
        
        sess.run(target, feed_dict={new_lr: lr, Dataplace: Data, Lbd: Lambda,
            tf_data_w: cur_w,
            tf_data_B: cur_B,
            tf_data_P: cur_P
        })

        if epoch % 1000 == 0:
            run_res = 0.0
            for times in range(100):
                run_res = run_res + sess.run(loss, feed_dict={Dataplace: Data, Lbd: Lambda,
                                               tf_data_w: cur_w,
                                               tf_data_B: cur_B,
                                               tf_data_P: cur_P})
            run_res = run_res/100.0
            print("epoch = %d, train error = %.4f" % (epoch + 1, run_res))

            if  run_res > prev_run_res:
                lr *= lr_decay
            prev_run_res = run_res

        epoch += 1
        
    cur_w, cur_B = sess.run([w, B])    

    third_grad_psi = np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))
    for r in range(1000):
        sess.run([tf_data_second])
        np.concatenate((third_grad_psi, np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))), axis=0) 

    u, s, vh = np.linalg.svd(np.matmul(np.transpose(third_grad_psi), third_grad_psi), full_matrices=True)
    O = u[:,0:k:1]
    print(s)
#    print(u)
#    print(O)
    cur_P = np.matmul(O, np.transpose(O))
#    print(cur_P)
    print(np.sum(np.square(third_grad_psi)))
    print(np.sum(np.square(third_grad_psi-np.matmul(third_grad_psi, cur_P))))
    print(np.sqrt(np.sum(np.square(P_correct-cur_P))))
    lib.plot.plot('Distance-till-correct', np.sqrt(np.sum(np.square(P_correct-cur_P))))
    lib.plot.plot('Total-variance-to-retain', np.sum(np.square(third_grad_psi-np.matmul(third_grad_psi, cur_P)))/np.sum(np.square(third_grad_psi)))
    lib.plot.plot('Total-variance-to-retain', np.sum(np.square(third_grad_psi-np.matmul(third_grad_psi, cur_P))))
    lib.plot.flush()
    lib.plot.tick()

sess.close()
print(cur_P)

epoch = 1, train error = 18515738.2000
epoch = 1001, train error = 5879639.8750
epoch = 2001, train error = 1265781.6875
epoch = 3001, train error = 214804.5056
epoch = 4001, train error = 32742.3024
epoch = 5001, train error = 4732.8444
epoch = 6001, train error = 671.2698
epoch = 7001, train error = 96.0938
epoch = 8001, train error = 13.9032
epoch = 9001, train error = 2.0144
epoch = 10001, train error = 0.2930
epoch = 11001, train error = 0.0435
epoch = 12001, train error = 0.0074
epoch = 13001, train error = 0.0022
epoch = 14001, train error = 0.0014
epoch = 15001, train error = 0.0013
epoch = 16001, train error = 0.0013
epoch = 17001, train error = 0.0013
epoch = 18001, train error = 0.0012
epoch = 19001, train error = 0.0013
epoch = 20001, train error = 0.0013
epoch = 21001, train error = 0.0013
epoch = 22001, train error = 0.0012
epoch = 23001, train error = 0.0012
epoch = 24001, train error = 0.0013
epoch = 25001, train error = 0.0013
epoch = 26001, train error = 0.0012
epoch 

epoch = 7001, train error = 0.0012
epoch = 8001, train error = 0.0012
epoch = 9001, train error = 0.0012
epoch = 10001, train error = 0.0013
epoch = 11001, train error = 0.0012
epoch = 12001, train error = 0.0013
epoch = 13001, train error = 0.0012
epoch = 14001, train error = 0.0012
epoch = 15001, train error = 0.0013
epoch = 16001, train error = 0.0013
epoch = 17001, train error = 0.0012
epoch = 18001, train error = 0.0013
epoch = 19001, train error = 0.0013
epoch = 20001, train error = 0.0012
epoch = 21001, train error = 0.0013
epoch = 22001, train error = 0.0012
epoch = 23001, train error = 0.0012
epoch = 24001, train error = 0.0013
epoch = 25001, train error = 0.0013
epoch = 26001, train error = 0.0012
epoch = 27001, train error = 0.0012
epoch = 28001, train error = 0.0012
epoch = 29001, train error = 0.0012
epoch = 30001, train error = 0.0013
epoch = 31001, train error = 0.0013
epoch = 32001, train error = 0.0013
epoch = 33001, train error = 0.0013
epoch = 34001, train error = 0.

epoch = 24001, train error = 0.0013
epoch = 25001, train error = 0.0012
epoch = 26001, train error = 0.0013
[9.18877333e-07 8.23783978e-07 7.94815946e-07 7.14175997e-07
 6.31226385e-07 5.61283457e-07 5.38942516e-07 5.19518096e-07
 4.80019139e-07 4.34853632e-07 2.29332979e-08 2.24677130e-08
 2.06009201e-08 2.05404618e-08 1.95771754e-08 1.86650944e-08
 1.79974631e-08 1.77635577e-08 1.74425647e-08 1.70430337e-08
 1.64752816e-08 1.57801843e-08 1.53788715e-08 1.52411754e-08
 1.46360319e-08 1.41436240e-08 1.38537501e-08 1.35463427e-08
 1.28480275e-08 1.23240538e-08 1.20941754e-08 1.14385976e-08
 1.10030349e-08 1.06788587e-08 1.04267297e-08 1.03251336e-08
 1.00220605e-08 9.48256140e-09 9.07099462e-09 8.66844641e-09
 8.31263325e-09 8.10000511e-09 7.62298580e-09 7.52678275e-09
 7.28412664e-09 6.93380686e-09 6.71839429e-09 6.41458575e-09
 6.24118179e-09 5.62629099e-09 3.78105602e-10 3.33502226e-10
 3.08758435e-10 2.92167013e-10 2.79349627e-10 2.72055323e-10
 2.66229705e-10 2.57154437e-10 2.35139

epoch = 1, train error = 0.0012
epoch = 1001, train error = 0.0012
epoch = 2001, train error = 0.0012
epoch = 3001, train error = 0.0013
epoch = 4001, train error = 0.0013
epoch = 5001, train error = 0.0013
epoch = 6001, train error = 0.0012
epoch = 7001, train error = 0.0013
epoch = 8001, train error = 0.0012
epoch = 9001, train error = 0.0013
epoch = 10001, train error = 0.0013
epoch = 11001, train error = 0.0013
epoch = 12001, train error = 0.0012
epoch = 13001, train error = 0.0012
epoch = 14001, train error = 0.0013
epoch = 15001, train error = 0.0012
epoch = 16001, train error = 0.0012
epoch = 17001, train error = 0.0012
epoch = 18001, train error = 0.0013
epoch = 19001, train error = 0.0012
epoch = 20001, train error = 0.0013
epoch = 21001, train error = 0.0012
epoch = 22001, train error = 0.0013
epoch = 23001, train error = 0.0012
epoch = 24001, train error = 0.0013
epoch = 25001, train error = 0.0013
epoch = 26001, train error = 0.0012
epoch = 27001, train error = 0.0013
epoch

epoch = 25001, train error = 0.0013
epoch = 26001, train error = 0.0012
epoch = 27001, train error = 0.0012
epoch = 28001, train error = 0.0012
epoch = 29001, train error = 0.0013
epoch = 30001, train error = 0.0013
epoch = 31001, train error = 0.0013
epoch = 32001, train error = 0.0012
epoch = 33001, train error = 0.0013
epoch = 34001, train error = 0.0013
epoch = 35001, train error = 0.0013
[7.90593958e-07 6.35422737e-07 6.24435700e-07 5.43979525e-07
 5.11382552e-07 4.60786453e-07 4.45521891e-07 3.85952319e-07
 3.57560282e-07 2.91723353e-07 1.15484227e-08 1.07060618e-08
 1.00899316e-08 9.71889769e-09 9.35587607e-09 9.13932396e-09
 8.77960282e-09 8.52082582e-09 8.39016856e-09 7.87724908e-09
 7.71904585e-09 7.41925232e-09 7.14865456e-09 7.00967906e-09
 6.84855150e-09 6.47250387e-09 6.30065466e-09 6.15216633e-09
 6.03627770e-09 5.79881521e-09 5.72148995e-09 5.61488456e-09
 5.37603251e-09 5.17003063e-09 5.03895103e-09 4.84420370e-09
 4.62750815e-09 4.48842252e-09 4.37771774e-09 4.1395460

...

epoch = 1, train error = 0.0013
epoch = 1001, train error = 0.0013
epoch = 2001, train error = 0.0012
epoch = 3001, train error = 0.0013
epoch = 4001, train error = 0.0012
epoch = 5001, train error = 0.0013
epoch = 6001, train error = 0.0012
epoch = 7001, train error = 0.0012
epoch = 8001, train error = 0.0013
epoch = 9001, train error = 0.0013
epoch = 10001, train error = 0.0012
epoch = 11001, train error = 0.0012
epoch = 12001, train error = 0.0013
epoch = 13001, train error = 0.0012
epoch = 14001, train error = 0.0012
epoch = 15001, train error = 0.0013
epoch = 16001, train error = 0.0012
epoch = 17001, train error = 0.0012
epoch = 18001, train error = 0.0012
epoch = 19001, train error = 0.0013
epoch = 20001, train error = 0.0013
epoch = 21001, train error = 0.0012
epoch = 22001, train error = 0.0013
[5.78372976e-07 5.34390551e-07 5.07504751e-07 4.11856178e-07
 4.05328933e-07 3.58647611e-07 3.31386815e-07 2.93664328e-07
 2.63451057e-07 2.13834952e-07 3.93518607e-10 3.52237017e-10
 3

epoch = 1, train error = 0.0012
epoch = 1001, train error = 0.0012
epoch = 2001, train error = 0.0013
epoch = 3001, train error = 0.0013
epoch = 4001, train error = 0.0012
epoch = 5001, train error = 0.0012
epoch = 6001, train error = 0.0013
epoch = 7001, train error = 0.0013
epoch = 8001, train error = 0.0013
epoch = 9001, train error = 0.0012
epoch = 10001, train error = 0.0012
epoch = 11001, train error = 0.0012
epoch = 12001, train error = 0.0012
epoch = 13001, train error = 0.0012
epoch = 14001, train error = 0.0012
epoch = 15001, train error = 0.0013
epoch = 16001, train error = 0.0013
epoch = 17001, train error = 0.0013
epoch = 18001, train error = 0.0013
epoch = 19001, train error = 0.0013
epoch = 20001, train error = 0.0013
epoch = 21001, train error = 0.0013
[5.81435472e-07 5.44167222e-07 4.72031076e-07 4.49459407e-07
 3.95511393e-07 3.75467295e-07 3.24925054e-07 3.10488417e-07
 2.91796596e-07 2.26284470e-07 4.01153888e-10 3.50785706e-10
 3.29631600e-10 2.90443863e-10 2.77401

epoch = 1, train error = 0.0013
epoch = 1001, train error = 0.0013
epoch = 2001, train error = 0.0012
epoch = 3001, train error = 0.0013
epoch = 4001, train error = 0.0013
epoch = 5001, train error = 0.0013
epoch = 6001, train error = 0.0012
epoch = 7001, train error = 0.0012
epoch = 8001, train error = 0.0013
epoch = 9001, train error = 0.0012
epoch = 10001, train error = 0.0012
epoch = 11001, train error = 0.0013
epoch = 12001, train error = 0.0013
epoch = 13001, train error = 0.0013
epoch = 14001, train error = 0.0012
epoch = 15001, train error = 0.0012
epoch = 16001, train error = 0.0012
epoch = 17001, train error = 0.0012
epoch = 18001, train error = 0.0013
epoch = 19001, train error = 0.0013
epoch = 20001, train error = 0.0013
epoch = 21001, train error = 0.0012
epoch = 22001, train error = 0.0012
epoch = 23001, train error = 0.0013
epoch = 24001, train error = 0.0012
epoch = 25001, train error = 0.0013
epoch = 26001, train error = 0.0013
epoch = 27001, train error = 0.0013
[5.75

epoch = 27001, train error = 0.0012
epoch = 28001, train error = 0.0013
epoch = 29001, train error = 0.0012
epoch = 30001, train error = 0.0013
[5.40966028e-07 5.02675960e-07 4.61741251e-07 4.28776048e-07
 4.00855470e-07 3.86059298e-07 3.21311433e-07 3.06172353e-07
 2.57824780e-07 2.04560450e-07 3.92335442e-10 3.78977849e-10
 3.36988853e-10 3.13446824e-10 2.85287738e-10 2.79724632e-10
 2.66417582e-10 2.52560972e-10 2.44178761e-10 2.37511760e-10
 2.25843996e-10 2.16611326e-10 2.11899651e-10 2.02979522e-10
 1.99317868e-10 1.93601163e-10 1.89834704e-10 1.80498311e-10
 1.71073863e-10 1.68591932e-10 1.65995939e-10 1.62651684e-10
 1.56062510e-10 1.54777427e-10 1.48739063e-10 1.46399337e-10
 1.42257428e-10 1.38993705e-10 1.37134110e-10 1.31839720e-10
 1.28866265e-10 1.24986049e-10 1.22153745e-10 1.19147053e-10
 1.17118523e-10 1.14306800e-10 1.10787025e-10 1.08940558e-10
 1.07467840e-10 1.03557454e-10 1.00801541e-10 9.79846898e-11
 9.63888483e-11 9.37600414e-11 8.82424342e-11 8.67587877e-11
 8

epoch = 1, train error = 0.0013
epoch = 1001, train error = 0.0013
epoch = 2001, train error = 0.0012
epoch = 3001, train error = 0.0012
epoch = 4001, train error = 0.0013
epoch = 5001, train error = 0.0013
epoch = 6001, train error = 0.0013
epoch = 7001, train error = 0.0013
epoch = 8001, train error = 0.0013
epoch = 9001, train error = 0.0012
